In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Chia dữ liệu thành các file riêng theo cluster và userid
========================================================
Lọc 2 file CSV (grades và logs) theo userid từ cluster_userids.json
và xuất ra các file riêng cho mỗi cluster và mỗi user.
"""

import json
import pandas as pd
from pathlib import Path
from typing import Dict, List

class DataSplitter:
    """Class để chia dữ liệu theo cluster"""
    
    def __init__(self, cluster_file: str, output_dir: str = None):
        """
        Parameters:
        -----------
        cluster_file : str
            Đường dẫn tới file cluster_userids.json
        output_dir : str
            Thư mục output (mặc định: cùng thư mục với cluster_file)
        """
        self.cluster_file = Path(cluster_file)
        self.output_dir = Path(output_dir) if output_dir else self.cluster_file.parent / "split_by_cluster"
        self.output_dir.mkdir(exist_ok=True, parents=True)
        
        # Load cluster userids
        with open(self.cluster_file, 'r') as f:
            self.cluster_data = json.load(f)
        
        # Tạo mapping ngược: userid -> cluster_id
        self.userid_to_cluster = {}
        for cluster_id, userids in self.cluster_data.items():
            for uid in userids:
                self.userid_to_cluster[uid] = cluster_id
        
        # All valid userids
        self.valid_userids = list(self.userid_to_cluster.keys())
        
        print(f"✓ Loaded {len(self.valid_userids)} userids from {len(self.cluster_data)} clusters")
        for cluster_id, userids in self.cluster_data.items():
            print(f"  - {cluster_id}: {len(userids)} users")
    
    def split_grades_file(self, grades_file: str):
        """
        Chia file grades (cấu trúc: id, timemodified, userid, courseid, finalgrade, itemtype)
        
        Parameters:
        -----------
        grades_file : str
            Đường dẫn tới file CSV grades
        """
        print(f"\n📊 Processing grades file: {grades_file}")
        
        # Load grades
        df = pd.read_csv(grades_file)
        print(f"   Total rows: {len(df)}")
        
        # Kiểm tra cột
        required_cols = ['userid']
        if not all(col in df.columns for col in required_cols):
            print(f"   ❌ Missing required columns. Found: {df.columns.tolist()}")
            return
        
        # Lọc theo valid userids
        df_filtered = df[df['userid'].isin(self.valid_userids)].copy()
        print(f"   Filtered rows: {len(df_filtered)} ({len(df_filtered)/len(df)*100:.1f}%)")
        
        # Thêm cột cluster
        df_filtered['cluster'] = df_filtered['userid'].map(self.userid_to_cluster)
        
        # 1. Xuất file tổng hợp theo cluster
        for cluster_id in self.cluster_data.keys():
            cluster_df = df_filtered[df_filtered['cluster'] == cluster_id]
            output_file = self.output_dir / f"grades_{cluster_id}.csv"
            cluster_df.drop('cluster', axis=1).to_csv(output_file, index=False)
            print(f"   ✓ Saved {output_file.name} ({len(cluster_df)} rows)")
        
        # 2. Xuất file riêng cho từng user
        user_dir = self.output_dir / "by_user" / "grades"
        user_dir.mkdir(exist_ok=True, parents=True)
        
        for userid in self.valid_userids:
            user_df = df_filtered[df_filtered['userid'] == userid]
            if len(user_df) > 0:
                cluster_id = self.userid_to_cluster[userid]
                output_file = user_dir / f"grades_user_{userid}_{cluster_id}.csv"
                user_df.drop('cluster', axis=1).to_csv(output_file, index=False)
        
        print(f"   ✓ Saved individual user files to {user_dir}/")
        
        # 3. Summary stats
        summary = df_filtered.groupby('cluster').agg({
            'userid': 'nunique',
            'id': 'count'
        }).rename(columns={'userid': 'n_users', 'id': 'n_records'})
        
        summary_file = self.output_dir / "grades_summary.csv"
        summary.to_csv(summary_file)
        print(f"   ✓ Saved summary to {summary_file.name}")
        print(f"\n   Summary:")
        print(summary.to_string())
        
        return df_filtered
    
    def split_logs_file(self, logs_file: str):
        """
        Chia file logs (cấu trúc: id, timecreated, eventname, action, target, userid, courseid, other)
        
        Parameters:
        -----------
        logs_file : str
            Đường dẫn tới file CSV logs
        """
        print(f"\n📋 Processing logs file: {logs_file}")
        
        # Load logs
        df = pd.read_csv(logs_file)
        print(f"   Total rows: {len(df)}")
        
        # Kiểm tra cột
        required_cols = ['userid']
        if not all(col in df.columns for col in required_cols):
            print(f"   ❌ Missing required columns. Found: {df.columns.tolist()}")
            return
        
        # Lọc theo valid userids
        df_filtered = df[df['userid'].isin(self.valid_userids)].copy()
        print(f"   Filtered rows: {len(df_filtered)} ({len(df_filtered)/len(df)*100:.1f}%)")
        
        # Thêm cột cluster
        df_filtered['cluster'] = df_filtered['userid'].map(self.userid_to_cluster)
        
        # Sắp xếp theo userid và timecreated
        if 'timecreated' in df_filtered.columns:
            df_filtered = df_filtered.sort_values(['userid', 'timecreated'])
        
        # 1. Xuất file tổng hợp theo cluster
        for cluster_id in self.cluster_data.keys():
            cluster_df = df_filtered[df_filtered['cluster'] == cluster_id]
            output_file = self.output_dir / f"logs_{cluster_id}.csv"
            cluster_df.drop('cluster', axis=1).to_csv(output_file, index=False)
            print(f"   ✓ Saved {output_file.name} ({len(cluster_df)} rows)")
        
        # 2. Xuất file riêng cho từng user
        user_dir = self.output_dir / "by_user" / "logs"
        user_dir.mkdir(exist_ok=True, parents=True)
        
        for userid in self.valid_userids:
            user_df = df_filtered[df_filtered['userid'] == userid]
            if len(user_df) > 0:
                cluster_id = self.userid_to_cluster[userid]
                output_file = user_dir / f"logs_user_{userid}_{cluster_id}.csv"
                user_df.drop('cluster', axis=1).to_csv(output_file, index=False)
        
        print(f"   ✓ Saved individual user files to {user_dir}/")
        
        # 3. Summary stats
        summary = df_filtered.groupby('cluster').agg({
            'userid': 'nunique',
            'id': 'count'
        }).rename(columns={'userid': 'n_users', 'id': 'n_events'})
        
        # Thêm top events
        if 'eventname' in df_filtered.columns:
            top_events = df_filtered.groupby('cluster')['eventname'].value_counts().groupby(level=0).head(5)
            print(f"\n   Top 5 events per cluster:")
            print(top_events.to_string())
        
        summary_file = self.output_dir / "logs_summary.csv"
        summary.to_csv(summary_file)
        print(f"\n   ✓ Saved summary to {summary_file.name}")
        print(f"\n   Summary:")
        print(summary.to_string())
        
        return df_filtered
    
    def create_report(self, grades_df=None, logs_df=None):
        """
        Tạo báo cáo tổng hợp
        """
        report = {
            'total_users': len(self.valid_userids),
            'clusters': {}
        }
        
        for cluster_id, userids in self.cluster_data.items():
            cluster_info = {
                'n_users': len(userids),
                'userids': userids
            }
            
            if grades_df is not None:
                cluster_grades = grades_df[grades_df['cluster'] == cluster_id]
                cluster_info['grades_records'] = len(cluster_grades)
            
            if logs_df is not None:
                cluster_logs = logs_df[logs_df['cluster'] == cluster_id]
                cluster_info['log_events'] = len(cluster_logs)
                
                if 'eventname' in cluster_logs.columns:
                    cluster_info['unique_event_types'] = cluster_logs['eventname'].nunique()
            
            report['clusters'][cluster_id] = cluster_info
        
        # Save report
        report_file = self.output_dir / "split_report.json"
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)
        
        print(f"\n✓ Saved split report to {report_file}")
        
        return report


def main():
    """Main function"""
    print("="*70)
    print("DATA SPLITTER BY CLUSTER")
    print("="*70)
    
    # Configuration
    CLUSTER_FILE = "../data/processed/cluster_userids.json"
    GRADES_FILE = "../data/udk_moodle_grades_course_670.filtered.csv"  # Thay đổi path nếu cần
    LOGS_FILE = "../data/udk_moodle_log_course_670.filtered.csv"      # Thay đổi path nếu cần
    OUTPUT_DIR = "../data/processed/split_by_cluster"
    
    # Initialize splitter
    splitter = DataSplitter(CLUSTER_FILE, OUTPUT_DIR)
    
    # Split grades file
    grades_df = None
    if Path(GRADES_FILE).exists():
        grades_df = splitter.split_grades_file(GRADES_FILE)
    else:
        print(f"\n⚠️  Grades file not found: {GRADES_FILE}")
    
    # Split logs file
    logs_df = None
    if Path(LOGS_FILE).exists():
        logs_df = splitter.split_logs_file(LOGS_FILE)
    else:
        print(f"\n⚠️  Logs file not found: {LOGS_FILE}")
    
    # Create report
    if grades_df is not None or logs_df is not None:
        report = splitter.create_report(grades_df, logs_df)
        
        print(f"\n{'='*70}")
        print("✅ SPLIT COMPLETE!")
        print("="*70)
        print(f"\nFiles created in: {splitter.output_dir}/")
        print(f"\nStructure:")
        print(f"  ├── grades_cluster_0.csv          # Grades for cluster 0")
        print(f"  ├── grades_cluster_1.csv          # Grades for cluster 1")
        print(f"  ├── logs_cluster_0.csv            # Logs for cluster 0")
        print(f"  ├── logs_cluster_1.csv            # Logs for cluster 1")
        print(f"  ├── grades_summary.csv            # Summary stats")
        print(f"  ├── logs_summary.csv              # Summary stats")
        print(f"  ├── split_report.json             # Complete report")
        print(f"  └── by_user/")
        print(f"      ├── grades/")
        print(f"      │   ├── grades_user_8609_cluster_0.csv")
        print(f"      │   └── ...")
        print(f"      └── logs/")
        print(f"          ├── logs_user_8609_cluster_0.csv")
        print(f"          └── ...")
    else:
        print(f"\n❌ No files processed!")


if __name__ == "__main__":
    main()


DATA SPLITTER BY CLUSTER
✓ Loaded 15 userids from 2 clusters
  - cluster_0: 12 users
  - cluster_1: 3 users

📊 Processing grades file: ../data/udk_moodle_grades_course_670.filtered.csv
   Total rows: 211
   Filtered rows: 211 (100.0%)
   ✓ Saved grades_cluster_0.csv (183 rows)
   ✓ Saved grades_cluster_1.csv (28 rows)
   ✓ Saved individual user files to ../data/processed/split_by_cluster/by_user/grades/
   ✓ Saved summary to grades_summary.csv

   Summary:
           n_users  n_records
cluster                      
cluster_0       12        183
cluster_1        3         28

📋 Processing logs file: ../data/udk_moodle_log_course_670.filtered.csv
   Total rows: 10833
   Filtered rows: 10833 (100.0%)
   ✓ Saved logs_cluster_0.csv (10093 rows)
   ✓ Saved logs_cluster_1.csv (740 rows)
   ✓ Saved individual user files to ../data/processed/split_by_cluster/by_user/logs/

   Top 5 events per cluster:
cluster    eventname                                   
cluster_0  \core\event\course_viewed  